# Financial Data Chatbot

This notebook builds a chatbot that answers questions from holdings and trades data. The bot only uses information from the provided CSV files. If a question cannot be answered from the data, it returns "Sorry can not find the answer".

Approach:
- Load holdings.csv and trades.csv
- Build a LangChain agent with pandas query tools
- The agent writes and executes pandas code to answer questions

## Setup

In [1]:
!pip install langchain langchain-openai pandas python-dotenv langchainhub -q


[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

## Load Data

In [3]:
holdings_df = pd.read_csv("data/holdings.csv")
trades_df = pd.read_csv("data/trades.csv")

print(f"Holdings: {holdings_df.shape[0]} rows, {holdings_df.shape[1]} columns")
print(f"Trades: {trades_df.shape[0]} rows, {trades_df.shape[1]} columns")

Holdings: 1022 rows, 25 columns
Trades: 649 rows, 31 columns


## Explore Holdings Data

In [4]:
holdings_df.head()

,AsOfDate,OpenDate,CloseDate,ShortName,PortfolioName,StrategyRefShortName,Strategy1RefShortName,Strategy2RefShortName,CustodianName,DirectionName,...,StartPrice,Price,StartFXRate,FXRate,MV_Local,MV_Base,PL_DTD,PL_QTD,PL_MTD,PL_YTD
0,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,568320.00,7.558656e+05,92.5040,10833.7294,92.5040,41054.5854
1,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,84.48,1.123584e+02,0.0138,1.6104,0.0138,6.1027
2,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,756000.00,1.005480e+06,123.0523,14411.4221,123.0523,54612.3074
3,01/08/23,04/03/20,NaN,Garfield,Garfield,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,484800.00,6.447840e+05,78.9097,9241.6104,78.9097,35021.2257
4,01/08/23,04/03/20,NaN,Heather,Heather,Default,Asset,DefaultS2,Well Prime,Long,...,96.0,96.0,1.33,1.33,487680.00,6.486144e+05,79.3785,9296.5110,79.3785,35229.2726


In [5]:
print("Holdings columns:")
print(holdings_df.columns.tolist())

Holdings columns:
['AsOfDate', 'OpenDate', 'CloseDate', 'ShortName', 'PortfolioName', 'StrategyRefShortName', 'Strategy1RefShortName', 'Strategy2RefShortName', 'CustodianName', 'DirectionName', 'SecurityId', 'SecurityTypeName', 'SecName', 'StartQty', 'Qty', 'StartPrice', 'Price', 'StartFXRate', 'FXRate', 'MV_Local', 'MV_Base', 'PL_DTD', 'PL_QTD', 'PL_MTD', 'PL_YTD']


In [6]:
print("Fund names in holdings (ShortName):")
print(holdings_df['ShortName'].unique())

Fund names in holdings (ShortName):
['Garfield' 'Heather' 'MNC Inv' 'NorthPoint' 'AIV 1' 'Opium' 'Ytum'
 'Platpot' 'Hi Yield' 'Lee Investment' 'SMA-L1' 'SMA-L2' 'SMA-L4'
 'IG Corp' 'Fund 2 LP' 'AIV 2' 'NPSMF3' 'NPSMF1' 'NPSMF2']


## Explore Trades Data

In [7]:
trades_df.head()

,id,RevisionId,AllocationId,TradeTypeName,SecurityId,SecurityType,Name,Ticker,CUSIP,ISIN,...,AllocationFees,AllocationCash,PortfolioName,CustodianName,StrategyName,Strategy1Name,Strategy2Name,Counterparty,AllocationRule,IsCustomAllocation
0,3489863,2,3460886,Buy,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,2800.00,7.002800e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,1
1,3489864,1,3460887,Sell,270471,Equity,Berry Brand 4/11 Equity,NaN,NaN,NaN,...,128.80,6.999871e+06,HoldCo 1,JP MORGAN SECURITIES LLC,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 1,0
2,3496826,1,3462756,Sell,290063,Equity,META-US,META,30303M102,US30303M1027,...,46985.99,2.553540e+09,HoldCo 3,CITIGROUP GLOBAL MARKETS INC.,Default,DefaultS1,DefaultS2,ABGS,Single Fund Rule - HoldCo 3,0
3,3496828,3,3462769,Buy,290067,Equity,SPOT-US,SPOT,NaN,LU1778762911,...,20.20,1.098249e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1
4,3496829,4,3462770,Buy,290067,Equity,SPOT-US,SPOT,NaN,LU1778762911,...,60.62,3.294749e+06,HoldCo 11,Goldman Sachs International,Default,Asset,DefaultS2,ABGS,Single Fund Rule - HoldCo 11,1


In [8]:
print("Trades columns:")
print(trades_df.columns.tolist())

Trades columns:
['id', 'RevisionId', 'AllocationId', 'TradeTypeName', 'SecurityId', 'SecurityType', 'Name', 'Ticker', 'CUSIP', 'ISIN', 'TradeDate', 'SettleDate', 'Quantity', 'Price', 'TradeFXRate', 'Principal', 'Interest', 'TotalCash', 'AllocationQTY', 'AllocationPrincipal', 'AllocationInterest', 'AllocationFees', 'AllocationCash', 'PortfolioName', 'CustodianName', 'StrategyName', 'Strategy1Name', 'Strategy2Name', 'Counterparty', 'AllocationRule', 'IsCustomAllocation']


In [9]:
print("Fund names in trades (PortfolioName):")
print(trades_df['PortfolioName'].unique())

Fund names in trades (PortfolioName):
['HoldCo 1' 'HoldCo 3' 'HoldCo 11' 'HoldCo 7' 'Redfield Accu-Fund'
 'UNC Investment Fund' 'ClientA' 'Leatherwood Trust MA' 'Platpot Fund'
 'Optimum Holdings Partners' 'Northpoint 401K' 'Account A' 'Account B'
 'Account D' 'Account C' 'CampNou Holdings']


## Key Observations

Looking at the data:
- Holdings uses `ShortName` for fund names (Garfield, Ytum, Platpot, etc.)
- Trades uses `PortfolioName` for fund names (HoldCo 1, ClientA, etc.)
- The fund names are different between the two tables
- Both tables share `SecurityId` and `CustodianName` which can be used to join them
- Holdings has P&L columns: PL_YTD, PL_QTD, PL_MTD, PL_DTD
- Holdings has market value in MV_Base (USD)

## Architecture

The chatbot uses a ReAct (Reasoning + Acting) agent pattern:

```
User Question → LLM → Thought → Action (Tool Call) → Observation → Answer
```

Components:
- **LLM**: GPT-4o-mini for fast, accurate responses
- **Tools**: Two pandas query tools (holdings_df, trades_df)
- **Agent**: ReAct agent that reasons about which tool to use
- **Executor**: Manages tool execution and error handling

The agent receives a question, decides which DataFrame to query, generates pandas code, executes it via the tool, and formulates a natural language answer.

In [10]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

SYSTEM_PROMPT = """You are a financial data analyst. Answer ONLY from the provided data.

DATA SCHEMA:

holdings_df (1,023 rows) - Portfolio positions. Each row = one holding. Funds have multiple holdings.
| Column | Description |
|--------|-------------|
| ShortName | Fund name |
| MV_Base | Market value (USD) |
| PL_YTD/QTD/MTD/DTD | Profit & Loss by period |
| SecurityId | Security ID (join key) |
| CustodianName | Broker (join key) |
| Qty, Price | Position details |

Funds: Garfield, Heather, MNC Inv, Ytum, Platpot, Opium, NorthPoint, Hi Yield, SMA-L1, SMA-L2, SMA-L4, Lee Investment, AIV 1, AIV 2, NPSMF1, NPSMF2, NPSMF3, IG Corp, Fund 2 LP

trades_df (649 rows) - Trade history. Each row = one trade.
| Column | Description |
|--------|-------------|
| PortfolioName | Fund name |
| TradeTypeName | Buy or Sell |
| SecurityId | Security ID (join key) |
| CustodianName | Broker (join key) |
| Quantity, Price, Principal | Trade details |

Funds: HoldCo 1, HoldCo 3, HoldCo 7, HoldCo 11, ClientA, UNC Investment Fund, Redfield Accu-Fund, Leatherwood Trust MA, Northpoint 401K, CampNou Holdings, Optimum Holdings Partners, Platpot Fund

CRITICAL RULES:
1. holdings_df uses ShortName, trades_df uses PortfolioName - DIFFERENT fund names
2. Join tables ONLY on SecurityId or CustodianName
3. Fund-level metrics require aggregation: df.groupby('ShortName')['column'].sum()
4. Pass raw pandas code to tools - no markdown, no backticks
5. Data not found → respond exactly: Sorry can not find the answer

EXAMPLES:
- Count holdings: holdings_df[holdings_df['ShortName'] == 'Garfield'].shape[0]
- Best YTD fund: holdings_df.groupby('ShortName')['PL_YTD'].sum().idxmax()
- Total trades: trades_df[trades_df['PortfolioName'] == 'HoldCo 1'].shape[0]
- Buy count: trades_df[trades_df['TradeTypeName'] == 'Buy'].shape[0]

Answer in ONE concise sentence. Format numbers with commas."""

In [11]:
@tool
def query_holdings_data(pandas_code: str) -> str:
    """Execute pandas code on holdings_df and return results."""
    try:
        namespace = {"holdings_df": holdings_df, "pd": pd, "result": None}
        code = pandas_code.strip()
        if code.startswith("```"):
            code = "\n".join(code.split("\n")[1:-1])
        if "result =" not in code:
            code = f"result = {code}"
        exec(code, {"__builtins__": {}}, namespace)
        result = namespace.get("result")
        return str(result) if result is not None else "No result"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def query_trades_data(pandas_code: str) -> str:
    """Execute pandas code on trades_df and return results."""
    try:
        namespace = {"trades_df": trades_df, "pd": pd, "result": None}
        code = pandas_code.strip()
        if code.startswith("```"):
            code = "\n".join(code.split("\n")[1:-1])
        if "result =" not in code:
            code = f"result = {code}"
        exec(code, {"__builtins__": {}}, namespace)
        result = namespace.get("result")
        return str(result) if result is not None else "No result"
    except Exception as e:
        return f"Error: {str(e)}"

tools = [query_holdings_data, query_trades_data]

In [12]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_react_agent(llm, tools, prompt=SYSTEM_PROMPT)

/var/folders/3j/k9pvdfpj7y78f4jsf6w3xp4c0000gn/T/ipykernel_4971/626496057.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools, prompt=SYSTEM_PROMPT)


In [13]:
def ask(question):
    messages = agent.invoke({"messages": [("user", question)]})
    return messages["messages"][-1].content

## Test the Chatbot

Running the example questions from the task.

### Question 1: Total number of holdings for a fund

In [14]:
ask("What is the total number of holdings for Garfield?")

'The total number of holdings for Garfield is 221.'

### Question 2: Total number of trades for a fund

In [15]:
ask("How many trades are there for HoldCo 1?")

'There are 43 trades for HoldCo 1.'

### Question 3: Which fund performed better based on yearly P&L

In [16]:
ask("Which fund has the best yearly profit and loss?")

'The fund with the best yearly profit and loss is Ytum.'

### Additional Questions

In [17]:
ask("What is the total market value for Platpot fund?")

'The total market value for Platpot fund is $890,828,500.'

In [18]:
ask("How many buy trades are there in total?")

'There are 504 buy trades in total.'

In [19]:
ask("List all the fund names in holdings")

'The fund names in holdings are: Garfield, Heather, MNC Inv, NorthPoint, AIV 1, Opium, Ytum, Platpot, Hi Yield, Lee Investment, SMA-L1, SMA-L2, SMA-L4, IG Corp, Fund 2 LP, AIV 2, NPSMF3, NPSMF1, NPSMF2.'

## Test Fallback Behavior

When asked about something not in the data, the bot should return "Sorry can not find the answer".

In [20]:
ask("Tell me about Bitcoin")

'Sorry can not find the answer.'

In [21]:
ask("What is the weather today?")

'Sorry can not find the answer.'

## Summary

The chatbot successfully answers questions from the holdings and trades data:
- Counts holdings and trades for specific funds
- Identifies top performing funds by yearly P&L
- Returns market values and other aggregations
- Correctly returns "Sorry can not find the answer" for questions outside the data

The agent uses a ReAct pattern to decide which data source to query and writes pandas code to extract the answer.